In [ ]:
import cv2
import numpy as np
import pandas as pd

# Load the video capture device (e.g., a file or camera index)
def load_video_capture(video_path):
    return cv2.VideoCapture(video_path)

# Define the person detection function using HOG+SVM
def detect_people(frame, hog):
    # Detect people in the frame
    boxes, weights = hog.detectMultiScale(
        frame,
        winStride=(4, 4),
        padding=(8, 8),
        scale=1.21
    )
    
    return boxes

# Define the person tracking function using Kalman filter
class KalmanFilter:
    def __init__(self):
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                                 [0, 1, 0, 1],
                                                 [0, 0, 1, 0],
                                                 [0, 0, 0, 1]])
        self.kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                                  [0, 1, 0, 0]])
        self.kalman.processNoiseCov = 1e-4 * np.eye(4)
        self.kalman.measurementNoiseCov = 1e-3 * np.eye(2)
        self.kalman.errorCovPost = 1e-1 * np.eye(4)
        self.kalman.statePost = np.zeros((4, 1))

    def predict(self):
        return self.kalman.predict()

    def update(self, measurement):
        self.kalman.correct(measurement)

# Define the main function
def count_unique_people(video_path):
    # Load the video capture device
    cap = load_video_capture(video_path)
    
    # Create a HOG+SVM person detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    # Initialize an empty list to store unique people
    unique_people = []
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect people in the frame
        boxes = detect_people(gray, hog)
        
        for box in boxes:
            x, y, w, h = box
            
            # Create a Kalman filter object
            kalman_filter = KalmanFilter()
            
            # Initialize the state with the current measurement
            measurement = np.array([[x + w / 2], [y + h / 2]])
            kalman_filter.update(measurement)
            
            # Predict the next state
            predicted_state = kalman_filter.predict()
            
            # Check if this person is already in the unique people list
            found = False
            for person in unique_people:
                if np.linalg.norm(predicted_state - person) < 50:
                    found = True
                    break
            
            if not found:
                unique_people.append(predicted_state)
        
        # Draw rectangles around detected people and display the frame
        for box in boxes:
            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        cv2.imshow('Frame', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    return len(unique_people)

# Test the function
video_path = 'path_to_your_video_file.mp4'
unique_count = count_unique_people(video_path)
print(f'Unique people count: {unique_count}')